In [1]:
from torchgeo.datasets import Sentinel2

data_dir = r"tests\data\sentinel2"

ds = Sentinel2(data_dir, bands=["B02", "B03", "B04", "B08"], cache=False, res=10)


Populating index


100%|██████████| 16/16 [00:00<00:00, 76.25it/s]


In [2]:
ds.bounds

BoundingBox(minx=399960.0, maxx=401240.0, miny=4498720.0, maxy=4500000.0, mint=1555079321.0, maxt=1649927271.999999)

In [3]:
from torchgeo.datasets.utils import BoundingBox
full_t_query = BoundingBox(minx=399960.0, maxx=400088.0, miny=4498720.0, maxy=4498848.0, mint=1555079321.0, maxt=1649927271.999999)
sample = ds[[full_t_query]]
print(sample["image"].shape)
print(sample["dates"])

torch.Size([1, 4, 13, 13])
[[datetime.datetime(2022, 4, 12, 16, 28, 41), datetime.datetime(2019, 4, 12, 16, 28, 41), datetime.datetime(2019, 4, 14, 11, 7, 51), datetime.datetime(2022, 4, 14, 11, 7, 51)]]


In [4]:
multi_t_query = [BoundingBox(minx=399960.0, maxx=400088.0, miny=4498720.0, maxy=4498848.0, mint=1555079321.0, maxt=1605264929),
                 BoundingBox(minx=399960.0, maxx=400088.0, miny=4498720.0, maxy=4498848.0, mint=1605264929.0, maxt=1649927272),
                 ]
sample = ds[multi_t_query]
print(sample["image"].shape)
print(sample["dates"])

torch.Size([2, 4, 13, 13])
[[datetime.datetime(2019, 4, 12, 16, 28, 41), datetime.datetime(2019, 4, 14, 11, 7, 51)], [datetime.datetime(2022, 4, 12, 16, 28, 41), datetime.datetime(2022, 4, 14, 11, 7, 51)]]


In [5]:
import rasterio
with rasterio.open(r"C:\Users\Sieger.Falkena\Documents\torchgeo\tests\data\sentinel2\S2A_MSIL2A_20220414T110751_N0400_R108_T26EMU_20220414T165533.SAFE\GRANULE\L2A_T26EMU_A035569_20220414T110747\IMG_DATA\R10m\T26EMU_20190414T110751_B02_10m.jp2") as src:
    print(src.nodata)

None
